In [9]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [10]:
orders = pd.read_csv('Orders_all_clean.csv', index_col=None, engine='python')

In [11]:
# remove the negative price orders either mistake or cancellation and currency symbols
orders['Total_Net_Price'] = orders['Total_Net_Price'].replace('[\$, ]', '', regex=True)  \
                           .replace('[\€, ]', '', regex=True).replace('[\�, ]', '', regex=True)  \
                           .replace('[R, ]', '', regex=True)
orders = orders[orders.Total_Net_Price != '(26.00)']
orders['Registration_Channel'] = orders['Registration_Channel'].fillna(value='Not Available').astype('category')

# Attributing sales source by registration type, creating new categorical series.
orders.loc[orders.Registration_Type == 'Speaker Guest', 'Sales_Source'] = 'Production'
orders.loc[orders.Registration_Type == 'Speaker', 'Sales_Source'] = 'Production'
orders.loc[orders.Registration_Type == 'Sponsor Attendee', 'Sales_Source'] = 'Sponsorship'
orders.Sales_Source.fillna(value='Marketing', inplace=True)
orders['Sales_Source_Cat'] = pd.Series()
orders.loc[orders.Registration_Type == 'Speaker Guest', 'Sales_Source_Cat'] = 'Production'
orders.loc[orders.Registration_Type == 'Speaker', 'Sales_Source_Cat'] = 'Production'
orders.loc[orders.Registration_Type == 'Sponsor Attendee', 'Sales_Source_Cat'] = 'Sponsorship'
orders.loc[orders.Registration_Type == 'Sponsor', 'Sales_Source_Cat'] = 'Sponsorship'
orders.loc[orders.Sales_Source == 'Marketing', 'Sales_Source_Cat'] = 'Marketing'
orders.Sales_Source_Cat.fillna(value='Delegate Sales', inplace=True)

In [12]:
# Creating unnique IDs for each person pased on first name, last name and email
# Original columns will be removed for data propection
orders['id'] = orders.groupby(['Contact_First_Name','Contact_Last_Name', 'Contact_Email_Address']).ngroup()

In [13]:
orders = orders.drop(columns=['Contact_First_Name','Contact_Last_Name', 'Contact_Email_Address'])

In [14]:
# Creating flags for group registrations - Group size variable indicates number in group of attendees

orders = orders[orders.Order_Number.notnull()]
orders['Order_Number']=orders['Order_Number'].astype(str)
orders['Group_Size'] = ""
dict = {}

for i in orders['Order_Number']:
    x = i[:12]
    if not x in dict:
        dict[x] = 1
    else:
        dict[x] += 1

group_size = []

for i in orders['Order_Number']:
    x = i[:12]
    group_size.append(dict[x])

orders['Group_Size'] = group_size

orders['Order_Number'].groupby(orders['Group_Size']).agg('count')

Group_Size
1     74573
2     13834
3      6864
4      3952
5      1930
6       996
7       476
8       472
9       234
10      260
11      132
12      144
13       91
14       14
15       30
17       17
20       20
21       42
Name: Order_Number, dtype: int64

In [15]:
#Creating a list of job title seniority based on full job titles. Appending list to DataFrame

orders['Contact_Job_Title'] = orders.Contact_Job_Title.fillna("UNKNOWN")

job_title = []
c_level = ['Chief','CEO','CMO','CDO','CIO','CPO','CTO','Head']
vp = ['VP','Vice President']

for i in orders['Contact_Job_Title']:
    if any(title in i for title in c_level):
        job_title.append("C-Level")
    elif any(title in i for title in vp):
        job_title.append("VP")
    elif "Director" in i:
        job_title.append("Director")
    elif "Manager" in i:
        job_title.append("Manager")
    elif "Consultant" in i:
        job_title.append("Consultant")
    else:
        job_title.append("Other")
        
orders['job_title'] = job_title
orders['job_title'].value_counts()

Director      25797
Manager       24560
Other         24074
VP            16218
C-Level       12349
Consultant     1083
Name: job_title, dtype: int64

In [16]:
orders.to_csv('orders_after_wrangling.csv')